# ADHD COACH CHATBOT

This notebook contains the code for a ADHD Chatbot



## Loading Data

In [1]:
# ADHD Coach Chatbot Project with TickTick Integration

# Note: This notebook will cover all steps and code components. When you're ready to implement it in Streamlit,
# transfer the code to a `.py` file and run it with `streamlit run your_app.py`.

# Import libraries
import requests
from requests_oauthlib import OAuth2Session
import streamlit as st
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [2]:
import requests
from requests_oauthlib import OAuth2Session
from http.server import BaseHTTPRequestHandler, HTTPServer
import webbrowser

# Replace these values with your app's information
client_id = "clientid"
client_secret = "clientsecret"
redirect_uri = "http://localhost:8010/callback"  # Ensure this matches your Todoist redirect URI
authorization_base_url = "https://todoist.com/oauth/authorize"
token_url = "https://todoist.com/oauth/access_token"

# Use the correct scope for read and write access
scope = "data:read_write"

# Step 1: Direct the user to Todoist for authorization with scope included
todoist = OAuth2Session(client_id, redirect_uri=redirect_uri, scope=[scope])
authorization_url, state = todoist.authorization_url(authorization_base_url)
print("Go to the following URL to authorize:", authorization_url)

# Open the authorization URL in the user's browser
webbrowser.open(authorization_url)

# Step 2: Define a helper function for the manual token exchange
def fetch_access_token(auth_code):
    # Manually exchange the authorization code for an access token
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "code": auth_code,
        "redirect_uri": redirect_uri,
    }
    response = requests.post(token_url, data=data)
    if response.status_code == 200:
        token = response.json()
        print("Access token retrieved:", token['access_token'])
        return token['access_token']
    else:
        print("Failed to retrieve access token:", response.status_code)
        print(response.json())  # Print error details for troubleshooting
        return None

# Step 3: Define the local server callback handler
class OAuthCallbackHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"You can close this window.")

        # Parse the authorization code from the URL
        query = self.path.split('?', 1)[-1]
        params = dict(qc.split('=') for qc in query.split('&'))
        authorization_code = params.get("code")

        if authorization_code:
            print("Authorization code received:", authorization_code)
            access_token = fetch_access_token(authorization_code)
            if access_token:
                self.server.token = {"access_token": access_token}
            else:
                self.server.token = None

# Step 4: Start the local server and handle the callback
server_address = ('', 8010)
httpd = HTTPServer(server_address, OAuthCallbackHandler)
print("Waiting for the callback with the authorization code...")
httpd.handle_request()

# Step 5: Retrieve the access token after successful authentication
if hasattr(httpd, 'token') and httpd.token:
    access_token = httpd.token['access_token']
    headers = {'Authorization': f"Bearer {access_token}"}
    print("Access token is set for further API calls.")
else:
    print("Failed to retrieve access token.")

Go to the following URL to authorize: https://todoist.com/oauth/authorize?response_type=code&client_id=5ce1f5a4f9e04772a9b7d336cac32656&redirect_uri=http%3A%2F%2Flocalhost%3A8010%2Fcallback&scope=data%3Aread_write&state=5XKmQWbrt3toGekiWVVvfRY0hPSGp6
Waiting for the callback with the authorization code...


127.0.0.1 - - [16/Oct/2024 09:04:39] "GET /callback?code=9da60c46c61bc5c5805b2950e6887e443e03a815&state=5XKmQWbrt3toGekiWVVvfRY0hPSGp6 HTTP/1.1" 200 -


Authorization code received: 9da60c46c61bc5c5805b2950e6887e443e03a815
Failed to retrieve access token: 400
{'error': 'invalid_grant'}
Failed to retrieve access token.


: 

In [3]:
from todoist_api_python.api import TodoistAPI

# Initialize the API client with your access token
api = TodoistAPI("access_token")

# Fetch tasks
try:
    tasks = api.get_tasks()  # Retrieves all active tasks
    for task in tasks:
        print(f"Task: {task.content} - Due: {task.due.date if task.due else 'No due date'}")
except Exception as error:
    print("Failed to fetch tasks:", error)


KeyboardInterrupt: 

: 

## Openai

In [4]:
import openai
print(openai.__version__)  

0.28.0


In [7]:

# Step 4: Setting Up OpenAI API for Chatbot
import openai

# Set your OpenAI API key
openai.api_key = "openai_api_key"
def generate_response(task, user_input):
    # Extract task details
    task_title = task.content if hasattr(task, 'content') else "Sample Task"
    task_description = task.description if hasattr(task, 'description') else "No description available"
    
    # Define the prompt
    prompt = f"You have a task: {task_title}. Details: {task_description}. {user_input}"
    
    # Call OpenAI's API to generate the response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

# Test the chatbot with a sample or fetched task
sample_task = tasks[0] if tasks else {'content': 'Sample Task'}
sample_input = "I need help getting started on this task"
print("Chatbot response:", generate_response(sample_task, sample_input))


Chatbot response: Sure, I can help you get started on your ADHD appointment task. Here are some steps you can take:

1. Schedule an appointment with a healthcare provider who specializes in ADHD, such as a psychiatrist or neurologist.
2. Gather any relevant medical records or information about your ADHD symptoms and history to bring to the appointment.
3. Make a list of any questions or concerns you have about your ADHD and treatment options.
4. Consider any specific goals or outcomes you would like to achieve from the appointment.
5. Set a reminder to attend the appointment on the scheduled date and time.

I hope this helps get you started on the task of preparing for your ADHD appointment. Let me know if you need further assistance.


In [8]:
import sys
print(sys.path)


['/Users/kripanavas/.pyenv/versions/3.11.3/lib/python311.zip', '/Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11', '/Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/lib-dynload', '', '/Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages']


## RAG

In [9]:
import pip
pip.main(["install", "tiktoken"])



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Requirement already satisfied: tiktoken in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (0.8.0)

Requirement already satisfied: regex>=2022.1.18 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from tiktoken) (2024.9.11)

Requirement already satisfied: requests>=2.26.0 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from tiktoken) (2.32.3)

Requirement already satisfied: charset-normalizer<4,>=2 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from requests>=2.26.0->tiktoken) (3.4.0)

Requirement already satisfied: idna<4,>=2.5 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from requests>=2.26.0->tiktoken) (3.10)

Requirement already satisfied: urllib3<3,>=1.21.1 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from requests>=2.26.0->tiktoken) (2.2.3)

Requirement already satisfied: certifi>=2017.4.17 in /Users/kripanavas/.pyenv/versions/3.11.3/lib/python3.11/site-packages (from requests>=2.26.0->tiktoken) (2024.8.30)

0

In [10]:
import sys
sys.path.insert(0, "/Users/kripanavas/Downloads/ADHD-Coach-main/.venv/lib/python3.11/site-packages")
import tiktoken
print(tiktoken.__version__)  # This should work if the path is correct


0.8.0


In [11]:
import os
import PyPDF2

In [6]:

# Step 5: RAG (Retrieval-Augmented Generation) Setup with ADHD Textbooks

import os
import PyPDF2

# Directory where your PDF files are stored
pdf_directory = "/Users/kripanavas/Downloads/ADHD-Coach-main/rag"  # Adjust the path based on where your PDFs are in VS Code

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_file):
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

# Extract text from all PDF files in the directory
texts = []
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(pdf_directory, filename)
        pdf_text = extract_text_from_pdf(file_path)
        texts.append(pdf_text)

# Combine all the extracted text
full_text = "\n\n".join(texts)

# Split the extracted text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.create_documents([full_text])


In [14]:
import os
os.environ["OPENAI_API_KEY"] = "openai_api_key"

In [16]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# Set up embeddings and vector store
embeddings = OpenAIEmbeddings(openai_api_key="openai_api_key")
vectorstore = FAISS.from_documents(documents, embeddings)

def retrieve_relevant_docs(query):
    # Retrieve relevant documents based on user query
    docs = vectorstore.similarity_search(query, k=5)  # Retrieve the top 5 relevant chunks
    return docs

def generate_coach_response_with_rag(task, user_input):
    # Retrieve relevant text from the textbooks
    relevant_docs = retrieve_relevant_docs(user_input)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])  # Extract content from each document
    
    # Combine context with user query and task information
    prompt = f"Use the following ADHD textbook information to help guide the user:\n\n{context}\n\n" \
             f"The user has the following task: {task['title']}. The user says: {user_input}"
    
    # Generate response from OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

# Example usage
task = {'title': 'graduation PPT'}
user_input = "I am having trouble focusing on this task, and I need some advice."
response = generate_coach_response_with_rag(task, user_input)
print(response)


Based on the information provided in the ADHD textbook, here are some tips that may help you improve your focus on preparing your graduation PPT:

- Break the task down into individual steps: Instead of trying to work on the entire presentation at once, break it down into smaller, manageable tasks. Start by outlining your main points, gathering relevant information, and creating a rough draft before polishing the final presentation.

- Gather what you need beforehand: Just like the examples of gathering all the ingredients before baking cookies, gather all the information, images, and resources you need for your presentation beforehand. This will help streamline the process and reduce distractions.

- Use cues and set up your environment: Before starting your task, think through the order of what you want to accomplish and create a conducive workspace. Consider listening to music or using a fidget to provide enough stimulation for your brain, so it doesn't get distracted.

- Practice n

In [19]:
def generate_response(user_input):
    few_shot_prompt = """
    The following are examples of ADHD coaching conversations:
    
    User: I can't focus on work.
    Kira: I understand. It might be challenging when concentration feels out of reach. Could we explore techniques to help you focus?
    
    User: I'm overwhelmed with my tasks.
    Kira: Breaking tasks into smaller steps might help. How about we start with one small part?
    
    User: """ + user_input
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": few_shot_prompt}],
        temperature=0.7,
        max_tokens=150
    )
    return response['choices'][0]['message']['content']
# Generate response using OpenAI API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.6,
        max_tokens=150
    )
    return response['choices'][0]['message']['content']
print(response)

Based on the information provided in the ADHD textbook, here are some tips that may help you improve your focus on preparing your graduation PPT:

- Break the task down into individual steps: Instead of trying to work on the entire presentation at once, break it down into smaller, manageable tasks. Start by outlining your main points, gathering relevant information, and creating a rough draft before polishing the final presentation.

- Gather what you need beforehand: Just like the examples of gathering all the ingredients before baking cookies, gather all the information, images, and resources you need for your presentation beforehand. This will help streamline the process and reduce distractions.

- Use cues and set up your environment: Before starting your task, think through the order of what you want to accomplish and create a conducive workspace. Consider listening to music or using a fidget to provide enough stimulation for your brain, so it doesn't get distracted.

- Practice n

In [20]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import webbrowser

In [ ]:

# Streamlit UI
st.title("Kira: Your ADHD Coach Chatbot")
st.write("Ask Kira for support with your tasks and ADHD-related guidance.")

# Input fields
task_title = st.text_input("What is your task?", placeholder="e.g., 'Complete my graduation PPT'")
user_input = st.text_input("What would you like help with?", placeholder="I need help focusing on this task")

# Process and display Kira's response when input is provided
if st.button("Ask Kira"):
    if task_title and user_input:
        response = generate_coach_response_with_rag(task_title, user_input)
        st.write(f"**Kira**: {response}")
    else:
        st.write("Please enter both a task and your question for Kira.")